In [220]:
from lib import read_file

from dataclasses import dataclass
from functools import reduce
import operator

import graphviz

In [317]:
@dataclass
class Workflow:
    decisions: list[str]

    def process(self, part: dict[str, int]) -> str:
        for d in self.decisions:
            print(d)
            if "<" in d or ">" in d:
                attr = d[0]
                val = part[attr]
                cond, dest = d.split(":")
                if "<" in d:
                    if val < int(cond.split("<")[-1]):
                        return dest
                elif ">" in d:
                    if val > int(cond.split(">")[-1]):
                        return dest
            else:
                return d

In [321]:
lines = read_file("19_input.txt")[::-1]
workflows = {}

l = lines.pop()
while len(lines):
    if l == "":
        break

    name, workflow = l.split("{")
    workflows[name] = Workflow(workflow.replace("}", "").split(","))
    l = lines.pop()

workflows

{'lx': Workflow(decisions=['a>1404:kq', 'a>736:hmc', 'hbb']),
 'db': Workflow(decisions=['a<2257:A', 's>3446:R', 'R']),
 'lhh': Workflow(decisions=['m<2267:zt', 'x>1979:A', 'vv']),
 'vgt': Workflow(decisions=['m<834:R', 'a>1057:R', 'a>1009:R', 'R']),
 'zfh': Workflow(decisions=['x<2428:gcf', 's>3274:qxr', 'hm']),
 'fvr': Workflow(decisions=['s<668:vgt', 'x>2691:A', 'vr']),
 'tgr': Workflow(decisions=['a<459:A', 'a<656:R', 'x>2162:R', 'A']),
 'lfp': Workflow(decisions=['x<2471:zm', 'fcx']),
 'psk': Workflow(decisions=['s<2112:gtm', 's>2147:A', 'x>2023:A', 'A']),
 'qxd': Workflow(decisions=['m<2102:R', 'x<112:A', 's<2655:R', 'A']),
 'vd': Workflow(decisions=['a>3406:A', 's>1912:R', 'x<2685:A', 'R']),
 'llc': Workflow(decisions=['m<865:R', 'x>941:A', 'A']),
 'vp': Workflow(decisions=['m<1653:R', 'vz']),
 'zn': Workflow(decisions=['s<2063:A', 'm<3691:A', 'A']),
 'vcz': Workflow(decisions=['a<979:R', 's<3048:A', 'm>1098:vx', 'R']),
 'tbc': Workflow(decisions=['a>3135:hsq', 'gkg']),
 'mqb': 

In [322]:
total = 0

for part in parts:
    workflow = "in"

    while True:
        workflow = workflows[workflow].process(part)
        if workflow == "R":
            break
        elif workflow == "A":
            total += sum(part.values())
            break

total

x<1853:qkm
x>798:pmz
vs
s<1883:st
x>491:kf
a>1463:tmf
x>624:std
a<780:hzt
x<689:bgq
rjm
s>3121:ndn
m<1642:sc
x>736:pzn
m<2625:R
A
x<1853:qkm
x>798:pmz
m>2665:tkr
a>1489:mqn
s<2459:tk
s<1211:nvc
s<645:sr
m<930:A
x<1853:qkm
qt
x>2842:jd
s<1418:gg
a<2229:fnv
s<2779:xm
a>1015:mhv
s<2272:ktc
a>548:lcx
x>2408:gvx
qtn
x>2115:dls
m<1847:gpz
a>362:A
R
x<1853:qkm
qt
x>2842:jd
s<1418:gg
a<1889:dm
m>1869:dqp
a>929:skf
x<2420:tp
kjs
a>350:bv
x>2618:R
a<553:R
x<1853:qkm
qt
x>2842:jd
s<1418:gg
a<1889:dm
m>2646:fv
s>859:xz
m>938:rjj
a>2955:vp
x<2194:lq
a<2246:mg
s<1092:A


18777

In [323]:
@dataclass
class Range:
    min: int
    max: int

    @property
    def n_vals(self):
        return max(0, self.max - self.min - 1)
    
    def overlapping_range(self, other: "Range"):
        # Return range constrained by both self and other
        if other.max <= self.min + 1:
            return None
        elif other.min >= self.max - 1:
            return None
        else:
            return Range(max(self.min, other.min), min(self.max, other.max))
        
    def subtract_range(self, other: "Range"):
        allowed = [Range(0, other.min + 1), Range(other.max - 1, 4001)]
        ret = [r.overlapping_range(self) for r in allowed]
        return [r for r in ret if r]


In [324]:
ranges = {c: Range(0, 4001) for c in "xmas"}
name = "in"
dot = graphviz.Digraph('round-table')  


def tot_ranges(ranges: dict[str, int]):
    tot = 1
    for range in ranges.values():
        tot *= range.n_vals
    return tot


seen = set()

def n_accepted(ranges: dict[str, int], workflow_name: str):
    if workflow_name not in ["R", "A"]:
        range_str = "\n".join([f"{key} {r.min} {r.max}" for key, r in ranges.items()])
        dot.node(workflow_name, workflow_name + "\n\n" + range_str) 
    
    if workflow_name == "R":
        return 0
    elif workflow_name == "A":
        return tot_ranges(ranges)
        
    def n_accepted_2(d_idx: int, ranges: dict[str, int]):
        decision = workflows[workflow_name].decisions[d_idx]

        if decision not in ["R", "A"]:
            dot.node(decision.replace(":", "="))
            dot.edge(workflow_name, decision.replace(":", "="))

        if decision == "R":
            return 0
        elif decision == "A":
            return tot_ranges(ranges)
        elif ":" not in decision:
            return n_accepted(ranges, decision)
        
        tot = 0
        cond, dest = decision.split(":")
        dot.edge(decision.replace(":", "="), dest)

        if "<" in cond:
            key, val = cond.split("<")
            r = Range(0, int(val))
        elif ">" in cond:
            key, val = cond.split(">")
            r = Range(int(val), 4001)
        else:
            assert False

        if (overlap := ranges[key].overlapping_range(r)):            
            # Accepted by this condition, send to next workflow
            ranges_copy = ranges.copy()
            ranges_copy[key] = overlap
            tot += n_accepted(ranges_copy, dest)

        for range in ranges[key].subtract_range(r):
            # Not accepted by this contidtion, send to next condition
            ranges_copy = ranges.copy()
            ranges_copy[key] = range
            tot += n_accepted_2(d_idx + 1, ranges_copy)

        return tot

    return n_accepted_2(d_idx=0, ranges=ranges)
        

n_accepted(ranges, "in")

121464316215623